In [1]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install torch-geometric

     |████████████████████████████████| 7.9 MB 4.0 MB/s 
     |████████████████████████████████| 3.5 MB 4.3 MB/s 
     |████████████████████████████████| 407 kB 2.7 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.4-py3-none-any.whl size=616603 sha256=a5cd6392cd9cf2fd99987db3169c461dd1fa0a93783c0df85a76723254d6888e
  Stored in directory: /root/.cache/pip/wheels/18/a6/a4/ca18c3051fcead866fe7b85700ee2240d883562a1bc70ce421
Successfully built torch-geometric


In [2]:
import torch
from sklearn.metrics import average_precision_score, roc_auc_score
from torch_geometric.utils import (add_self_loops, negative_sampling,
                                   remove_self_loops)
EPS = 1e-15
MAX_LOGSTD = 10

class InnerProductDecoder(torch.nn.Module):
    r"""The inner product decoder from the `"Variational Graph Auto-Encoders"
    <https://arxiv.org/abs/1611.07308>`_ paper

    .. math::
        \sigma(\mathbf{Z}\mathbf{Z}^{\top})

    where :math:`\mathbf{Z} \in \mathbb{R}^{N \times d}` denotes the latent
    space produced by the encoder."""
    def forward(self, z, edge_index, sigmoid=True):
        r"""Decodes the latent variables :obj:`z` into edge probabilities for
        the given node-pairs :obj:`edge_index`.

        Args:
            z (Tensor): The latent space :math:`\mathbf{Z}`.
            sigmoid (bool, optional): If set to :obj:`False`, does not apply
                the logistic sigmoid function to the output.
                (default: :obj:`True`)
        """
        value = (z[edge_index[0]] * z[edge_index[1]]).sum(dim=1)
        return torch.sigmoid(value) if sigmoid else value


    def forward_all(self, z, sigmoid=True):
        r"""Decodes the latent variables :obj:`z` into a probabilistic dense
        adjacency matrix.

        Args:
            z (Tensor): The latent space :math:`\mathbf{Z}`.
            sigmoid (bool, optional): If set to :obj:`False`, does not apply
                the logistic sigmoid function to the output.
                (default: :obj:`True`)
        """
        adj = torch.matmul(z, z.t())
        return torch.sigmoid(adj) if sigmoid else adj



class GAE(torch.nn.Module):
    r"""The Graph Auto-Encoder model from the
    `"Variational Graph Auto-Encoders" <https://arxiv.org/abs/1611.07308>`_
    paper based on user-defined encoder and decoder models.

    Args:
        encoder (Module): The encoder module.
        decoder (Module, optional): The decoder module. If set to :obj:`None`,
            will default to the
            :class:`torch_geometric.nn.models.InnerProductDecoder`.
            (default: :obj:`None`)
    """
    def __init__(self, encoder, decoder=None):
        super().__init__()
        self.encoder = encoder
        self.decoder = InnerProductDecoder() if decoder is None else decoder
    

    def encode(self, *args, **kwargs):
        r"""Runs the encoder and computes node-wise latent variables."""
        return self.encoder(*args, **kwargs)


    def decode(self, *args, **kwargs):
        r"""Runs the decoder and computes edge probabilities."""
        return self.decoder(*args, **kwargs)


    def recon_loss(self, z, pos_edge_index, neg_edge_index=None):
        r"""Given latent variables :obj:`z`, computes the binary cross
        entropy loss for positive edges :obj:`pos_edge_index` and negative
        sampled edges.

        Args:
            z (Tensor): The latent space :math:`\mathbf{Z}`.
            pos_edge_index (LongTensor): The positive edges to train against.
            neg_edge_index (LongTensor, optional): The negative edges to train
                against. If not given, uses negative sampling to calculate
                negative edges. (default: :obj:`None`)
        """

        pos_loss = -torch.log(
            self.decode(z)[pos_edge_index[0],pos_edge_index[1]] + EPS).mean()

        # Do not include self-loops in negative samples
        pos_edge_index, _ = remove_self_loops(pos_edge_index)
        pos_edge_index, _ = add_self_loops(pos_edge_index)
        if neg_edge_index is None:
            neg_edge_index = negative_sampling(pos_edge_index, z.size(0))
        neg_loss = -torch.log(1 -
                              self.decode(z)[neg_edge_index[0],neg_edge_index[1]] +
                              EPS).mean()

        return pos_loss + neg_loss

    #define test function based on decoder
    def test(self, z, pos_edge_index, neg_edge_index):
        r"""Given latent variables :obj:`z`, positive edges
        :obj:`pos_edge_index` and negative edges :obj:`neg_edge_index`,
        computes area under the ROC curve (AUC) and average precision (AP)
        scores.

        Args:
            z (Tensor): The latent space :math:`\mathbf{Z}`.
            pos_edge_index (LongTensor): The positive edges to evaluate
                against.
            neg_edge_index (LongTensor): The negative edges to evaluate
                against.
        """
        from sklearn.metrics import average_precision_score, roc_auc_score

        pos_y = z.new_ones(pos_edge_index.size(1))
        neg_y = z.new_zeros(neg_edge_index.size(1))
        y = torch.cat([pos_y, neg_y], dim=0)

        pred_test_neg=model.decode(z)[data.test_neg_edge_index[0],data.test_neg_edge_index[1]]
        pred_test_pos=model.decode(z)[data.test_pos_edge_index[0],data.test_pos_edge_index[1]]
        pred=torch.cat([pred_test_pos,pred_test_neg],dim=0)

        y, pred = y.detach().cpu().numpy(), pred.detach().cpu().numpy()
  
        return roc_auc_score(y, pred), average_precision_score(y, pred)



class VGAE(GAE):
    r"""The Variational Graph Auto-Encoder model from the
    `"Variational Graph Auto-Encoders" <https://arxiv.org/abs/1611.07308>`_
    paper.

    Args:
        encoder (Module): The encoder module to compute :math:`\mu` and
            :math:`\log\sigma^2`.
        decoder (Module, optional): The decoder module. If set to :obj:`None`,
            will default to the
            :class:`torch_geometric.nn.models.InnerProductDecoder`.
            (default: :obj:`None`)
    """
    def __init__(self, encoder, decoder=None):
        super().__init__(encoder, decoder)

    def reparametrize(self, mu, logstd):
        if self.training:
            return mu + torch.randn_like(logstd) * torch.exp(logstd)
        else:
            return mu


    def encode(self, *args, **kwargs):
        """"""
        self.__mu__, self.__logstd__ = self.encoder(*args, **kwargs)
        self.__logstd__ = self.__logstd__.clamp(max=MAX_LOGSTD)
        z = self.reparametrize(self.__mu__, self.__logstd__)
        return z


    def kl_loss(self, mu=None, logstd=None):
        r"""Computes the KL loss, either for the passed arguments :obj:`mu`
        and :obj:`logstd`, or based on latent variables from last encoding.

        Args:
            mu (Tensor, optional): The latent space for :math:`\mu`. If set to
                :obj:`None`, uses the last computation of :math:`mu`.
                (default: :obj:`None`)
            logstd (Tensor, optional): The latent space for
                :math:`\log\sigma`.  If set to :obj:`None`, uses the last
                computation of :math:`\log\sigma^2`.(default: :obj:`None`)
        """
        mu = self.__mu__ if mu is None else mu
        logstd = self.__logstd__ if logstd is None else logstd.clamp(
            max=MAX_LOGSTD)
        return -0.5 * torch.mean(
            torch.sum(1 + 2 * logstd - mu**2 - logstd.exp()**2, dim=1))

In [3]:
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges
import torch.nn as nn
import torch.nn.init as init
from torch.utils.tensorboard import SummaryWriter
from torch.autograd import Variable

In [4]:
dataset = Planetoid("\..", "CiteSeer", transform=T.NormalizeFeatures())

Processing...
Done!


In [5]:
data = dataset[0]
data.train_mask = data.val_mask = data.test_mask = data.y = None
data = train_test_split_edges(data)

/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [6]:
data

Data(x=[3327, 3703], val_pos_edge_index=[2, 227], test_pos_edge_index=[2, 455], train_pos_edge_index=[2, 7740], train_neg_adj_mask=[3327, 3327], val_neg_edge_index=[2, 227], test_neg_edge_index=[2, 455])

In [7]:
from torch_geometric.utils import to_dense_adj

In [8]:
class VariationalGCNEncoder(torch.nn.Module):
    def __init__(self,embedding_size,in_channels, out_channels):
        super(VariationalGCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * embedding_size, cached=True) # cached only for transductive learning
        self.conv_mu = GCNConv(2 * embedding_size, out_channels, cached=True)
        self.conv_logstd = GCNConv(2 * embedding_size, out_channels, cached=True)

    def forward(self, x, edge_index):
        hidden = self.conv1(x, edge_index).relu()

        z_mu=self.conv_mu(hidden, edge_index)
        z_lsgms=self.conv_logstd(hidden, edge_index)
        return z_mu,z_lsgms

In [9]:
# definition of terms
# h: hidden state of LSTM
# y: edge prediction, model output
# n: noise for generator
# l: whether an output is real or not, binary
import torch.nn.functional as F
# plain LSTM model
class RNN_plain(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers,output_size):
        super(RNN_plain, self).__init__()
        torch.manual_seed(1)

        self.num_layers = num_layers
        self.hidden_size = hidden_size
        #self.batch_size=batch_size


        #self.input = nn.Linear(input_size, embedding_size)
        #self.linear_input = nn.Linear(input_size, embedding_size)

        self.rnn = nn.RNN(input_size=embedding_size, hidden_size=hidden_size, num_layers=num_layers)

        self.output = nn.Sequential(
                nn.Linear(hidden_size, output_size),
                nn.ReLU()
                #nn.Linear(4*hidden_size, 8*hidden_size),
                #nn.ReLU(),
                #nn.Linear(8*hidden_size,output_size)
           )

        self.relu = nn.ReLU()
       
        #self.tanh = nn.Tanh()
        #self.dropout= nn.Dropout(p=0.3)
        # initialize

        self.hidden = None

        for name, param in self.rnn.named_parameters():
            if 'bias' in name:
                nn.init.constant(param, 0.25)
            elif 'weight' in name:
                nn.init.xavier_uniform(param,gain=nn.init.calculate_gain('sigmoid'))
        for m in self.modules():
            if isinstance(m, nn.Linear):
                m.weight.data = init.xavier_uniform(m.weight.data, gain=nn.init.calculate_gain('relu'))

    def init_hidden(self):
        return torch.zeros(1,1, self.hidden_size).cuda()

    def forward(self, z):
        #input = self.relu(self.input(z))
        #input = F.dropout(input, p=0.2)
        #input = self.dropout(input)
        #input = self.relu(input)
        #linear shape torch.Size([3327, 512])
        h0 = self.init_hidden()
        #print(h0.shape)
        input=z.unsqueeze(1)
        #print(input.shape)
        output_raw, h_out = self.rnn(input, h0)
        #print(output_raw.shape)
        #output_raw = self.tanh(output_raw)
        #lstm shape torch.Size([1, 3327, 1024]
        output_raw = self.output(output_raw.view(-1, output_raw.size(2)))
        #output_raw = F.dropout(output_raw, p=0.2)

        # return hidden state at each time step
        #output_raw=output_raw.view(-1, output_raw.size(2))
        #linear shape torch.Size([3327, 3327])
        return torch.sigmoid(output_raw)

In [10]:
#out_channels = 2
num_features = dataset.num_features
epochs = 300
embedding_size=64
hidden_size=256
num_layers=1


In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)
model=VGAE(VariationalGCNEncoder(embedding_size=embedding_size,in_channels=num_features, out_channels=embedding_size),
           decoder=RNN_plain(input_size=embedding_size, embedding_size=embedding_size,hidden_size=hidden_size, num_layers=num_layers,output_size=3327))
model = model.to(device)
#test_pos_edge_index=data.test_pos_edge_index.to(device)
#test_neg_edge_index=data.test_neg_edge_index.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


In [12]:
model

VGAE(
  (encoder): VariationalGCNEncoder(
    (conv1): GCNConv(3703, 128)
    (conv_mu): GCNConv(128, 64)
    (conv_logstd): GCNConv(128, 64)
  )
  (decoder): LSTM_plain(
    (rnn): RNN(64, 256)
    (output): Sequential(
      (0): Linear(in_features=256, out_features=3327, bias=True)
      (1): ReLU()
    )
    (relu): ReLU()
  )
)

In [13]:
data.x.size(0)

3327

In [14]:
data['x'].float().size(0)

3327

In [15]:
def train():
    model.train()
    optimizer.zero_grad()
    #model.decoder.hidden=model.decoder.init_hidden(batch_size=data['x'].float())
    z = model.encode(x.to(device), train_pos_edge_index.to(device))
    loss = model.recon_loss(z, train_pos_edge_index)

    loss =  (1 / data.num_nodes) * model.kl_loss()  # new line

    loss.backward()
    optimizer.step()
    print("loss:",float(loss))
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z,pos_edge_index, neg_edge_index)

In [16]:
#from torch.utils.tensorboard import SummaryWriter
#writer = SummaryWriter('runs/VGAE+LSTM_experiment_'+'2d_100_epochs')

In [17]:
for epoch in range(1, epochs):
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d},AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

    #writer.add_scalar('loss train',loss,epoch)
    #writer.add_scalar('auc train',auc,epoch) # new line
    #writer.add_scalar('ap train',ap,epoch)   # new line

loss: 1.4662460046110937e-07
Epoch: 001,AUC: 0.4747, AP: 0.4816
loss: 3.546807647580863e-06
Epoch: 002,AUC: 0.4748, AP: 0.4817
loss: 3.418600442728348e-07
Epoch: 003,AUC: 0.4755, AP: 0.4821
loss: 4.6456875679723453e-07
Epoch: 004,AUC: 0.4760, AP: 0.4826
loss: 1.3532331877286197e-06
Epoch: 005,AUC: 0.4760, AP: 0.4826
loss: 1.2235166195750935e-06
Epoch: 006,AUC: 0.4765, AP: 0.4822
loss: 5.785431085314485e-07
Epoch: 007,AUC: 0.4760, AP: 0.4825
loss: 1.9181159416348237e-07
Epoch: 008,AUC: 0.4753, AP: 0.4824
loss: 2.658650259945716e-07
Epoch: 009,AUC: 0.4753, AP: 0.4823
loss: 5.043194164500164e-07
Epoch: 010,AUC: 0.4752, AP: 0.4822
loss: 5.829413680658035e-07
Epoch: 011,AUC: 0.4747, AP: 0.4823
loss: 4.5270510895534244e-07
Epoch: 012,AUC: 0.4747, AP: 0.4820
loss: 2.6671600039662735e-07
Epoch: 013,AUC: 0.4753, AP: 0.4823
loss: 1.685663022499284e-07
Epoch: 014,AUC: 0.4753, AP: 0.4820
loss: 1.8214620922663016e-07
Epoch: 015,AUC: 0.4754, AP: 0.4822
loss: 2.3769300128151372e-07
Epoch: 016,AUC: 0.